# DOES NOT YET GIVE SAME RESULTS AS CORRECT PARKING_COST_APRIL22.IPYNB


# Estimating Parking Cost and Spatial Autocorrelation Analysis of Parking Data

Goals:
   1. Join csv cost data with spatial data for parking lots
   3. Estimate Ratios of M to D, D to H, M to H to estimate missing rates values.
   4. KNN for points to polygons (TAZs)

For inflation adjustment: https://www.inflationtool.com/us-dollar/2010-to-present-value    

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import pysal
from osgeo import gdal
import copy
import libpysal as lps
import scipy
from itertools import combinations

## Bring In Data

1. Lot Rates
2. Lot Points
3. Join Points and Rates
4. Filter Lots (must have at least one rate)
5. TAZs

In [2]:
# bring in data
base = "J:\\Shared drives\\TMD_TSA\\Data\\Parking\\WebScraped_ParkingCost\\required_inputs"
# parking costs
rates = pd.read_csv(base+"\parking_cost_fullrec_NAP_F16.csv")

# spatial points
points = gpd.read_file(base+"\GeocodedParkingLots\DKedits_parking_cost_fullrec_NAP.shp")
points = points.dropna(subset=["geometry"])

# join cost to points
lots = points[['IN_SingleL','geometry','USER_month','USER_lot_u']].merge(rates[['IN_SingleLine','USER_lot_url',
                                                                                'MR','DR','HR']],
                                                                         left_on='USER_lot_u',right_on='USER_lot_url')


In [3]:
# bring in relevant TAZs
base2 = "J:\Shared drives\TMD_TSA\Data\GIS Data\TAZ"
alltazs = gpd.read_file(base2+"\\candidate_CTPS_TAZ_STATEWIDE_2019_wgs84.shp")
# filter to just relevant municipalities
#alltazs = alltazs[(alltazs['town'].isin(["BOSTON","CAMBRIDGE","SOMERVILLE","BROOKLINE","NEWTON"])) & (alltazs['id'] < 200000)][["id","town","geometry"]]

## Estimate and Fill Missing Monthly Rates

Calculate the Monthly/Daily ratio per district by dividing the monthly column by the daily column to get lot level ratios and aggregate to the region. This region-wide ratio is multiplied by each lot's Daily Rate to calculate an estimated monthly rate. At this point, a new column is made where observed monthly rate data unless missing, then estimated monthly rate data is used if existing (aka if lot has a daily rate). 

This will be conducted for M/D, D/H, and M/H - M/D is used as the example for the explanation for ease of understanding.

In [4]:
def estimate_lot_rates(tps, lots):
    # filter out customer only parking (no rates for any category)
    estmonth = copy.deepcopy(lots[lots[tps].notna().any(axis='columns')])
    #estmonth = copy.deepcopy(lots[(~lots['MR'].isna()) | (~lots['DR'].isna()) | (~lots['HR'].isna())])
    
    tp2_0 = []
    # estimate round 1: lot rate estimations based on other tp values
    for tp2 in list(combinations(tps, 2)):
        if 'Est_'+tp2[0] in estmonth.columns:
            tp2 = tuple(reversed(tp2))
            if 'Est_'+tp2[0] in estmonth.columns:
                continue # if both are already estimated go to next pair
            else:
                pass
        else:
            pass
        ratio = tp2[0] +'_to_'+tp2[1]
        
        #get ratio at the lot level
        estmonth[ratio] = np.where((estmonth[tp2[1]]== 0) | (estmonth[tp2[0]]== 0), 0,estmonth[tp2[0]]/estmonth[tp2[1]])           
        
        # estimate monthly from daily and mean regional ratio (using only where both values)
        estmonth['Est_'+tp2[0]] = estmonth[tp2[1]] * estmonth[ratio].mean()

        # combine estimated daily with actual daily where possible
        estmonth[tp2[0]+'_wEst'] = np.where(estmonth[tp2[0]].isna(),
                                                 estmonth['Est_'+tp2[0]],
                                                 estmonth[tp2[0]])
    
    # estimate round 2: lot rate estimates based on other estimated values
    tp2_0 = []
    for tp2 in list(combinations(tps, 2)):
        if 'Est_'+tp2[0]+'_2' in estmonth.columns:
            tp2 = tuple(reversed(tp2))
            if 'Est_'+tp2[0]+'_2' in estmonth.columns:
                continue # if both are already re-estimated go to next pair
        tp2_0.append(tp2[0])
        estmonth['Est_'+tp2[0]+'_2'] = estmonth[tp2[1]+'_wEst'] * estmonth[tp2[0] +'_to_'+tp2[1]].mean()

        # combine estimated daily with actual daily where possible
        estmonth[tp2[0]+'_wEst2'] = np.where(estmonth[tp2[0]].isna(),
                                                 estmonth['Est_'+tp2[0]+'_2'],
                                                 estmonth[tp2[0]])
    
    return estmonth

In [5]:
tps = ["MR", "DR", "HR"]
K = 16
all_lots = lots
all_tazs = alltazs


elots = estimate_lot_rates(tps, all_lots)

# post LM geojson - only need if running remove_outliers
postSAdf = gpd.read_file(base+"\estmonth_April14_HR_DR_MR_LM.geojson")
cluster_outlier_field = "COType_"


In [23]:
#EXPORTS

elots.to_csv("J:\Shared drives\TMD_TSA\Data\Parking\WebScraped_ParkingCost\estmonth_April25d.csv")

# Export estmonth as geojson for use in ArcPro and QGIS for Local Moran's I and Getis Ord Gi*
elots.to_file("J:\Shared drives\TMD_TSA\Data\Parking\WebScraped_ParkingCost\estmonth_April25d.geojson") 

# Calculate Average Rates per TAZ
### Post Local Spatial Autocorrelation (Local Moran's I)
The data is clustered - see versions of this analysis with Spatial Autocorrelation included for information on that.

In [8]:
def remove_outliers(lots, postSA, cof):
    # if importing new data to add to lots, add and delete outliers
    if isinstance(postSA, gpd.GeoDataFrame) and (len(cof) > 0): # if importing new data with LM outliers
        postSA = postSA.to_crs(26986)
        estmonth = lots.sjoin_nearest(postSA[[cluster_outlier_field+s for s in tps]+["geometry"]], how="left")
        
        # 1 and 13 are very close to each other (see index_right) removing them so can filter later
        estmonth = estmonth[~estmonth.index.duplicated(keep='first')]
        
        # get lot ids where HL or LH for each time period and exclude them from the weighted average
        inin =[]
        for x in tps:
            inin.append(estmonth[~estmonth[cof+x].isin(["LH", "HL"])].reset_index()['index']) 
    return estmonth, inin

In [9]:
def euclidean_matrix(lots,tazs,K,tps,inin=[]):

    estmonth = copy.deepcopy(lots)
    if len(inin) == 0:
        for x in tps:
            inin.append(estmonth.reset_index['index'])
    else:
        pass

    # get euclidean distance matrix from TAZ centroids to lots
    # also reproject to Mass State Plane (meters) so that distance is correct
    rdg83 = alltazs.to_crs("EPSG:26986").set_index("id") # TAZ ids are now the column names
    estmonth =  estmonth.to_crs("EPSG:26986")
    eucdist = estmonth.centroid.geometry.apply(lambda g: rdg83.distance(g))
    # convert to miles
    eucdistmi = eucdist/1609.34
    
    # get just closest 16 lots to each TAZ centroid based on euclidean distance
    numlot = len(eucdistmi)
    for col in eucdistmi.columns:
        big = max(eucdistmi[col].nsmallest(K))
        eucdistmi.loc[eucdistmi[col] > big, col]= np.nan

    # set distances (weights) to 1 so all have equal weights
    eucdistmi[eucdistmi.notna()] = 1
    
    return eucdistmi, inin

In [20]:
def knn_average(tps, estmonth, eucdistmi, inin, tazs):
    # tazs & lots to filter
    tazids = tazs[(tazs['town'].isin(["BOSTON","CAMBRIDGE","SOMERVILLE",
                                        "BROOKLINE","NEWTON"])) & (tazs['id'] < 200000)]["id"].tolist()
    

    # multiply weights (1) by rates, reminder: weights are all 1 so this is essentially a mask
    # filter the rates and weights by whether the lot is an outlier, then multiply
    msums = []
    for z in tps:
        g = estmonth[z+'_wEst2'].filter(items = inin[tps.index(z)], axis=0)
        m = eucdistmi.filter(items = inin[tps.index(z)], axis=0).multiply(g, axis="index")
        msum = m.sum()
        msum.name = z+"_SumNN"
        #save so can merge together later
        msums.append(msum)
        
    #sum weights by TAZ
    wsum = eucdistmi.sum() # should be K from KNN
    wsum.name = "TotalNN"
    
    # join weighted rates sums by taz and sum weights by taz together
    for q in msums:
        wsum = pd.merge(wsum,q, left_index=True, right_index=True)
        wsum[q.name] = np.where(~wsum.index.isin(tazids), 0, wsum[q.name])
        wsum[q.name.split("_")[0]+"_Avg_NN"] = wsum[q.name]/wsum["TotalNN"]
        
        # Convert to 2010
        wsum[q.name.split("_")[0]+"_Avg_NN_2010"] = wsum[q.name.split("_")[0]+"_Avg_NN"] * 0.69
        
    # final spatial result    
    tazs_avg_rates = pd.merge(tazs.set_index("id"),wsum, left_index=True, right_index=True)
    
    return tazs_avg_rates


In [17]:
noOutlots, noOutlist = remove_outliers(elots, postSAdf,cluster_outlier_field)

In [18]:
distmatrix, noOutlist = euclidean_matrix(noOutlots,all_tazs,K,tps,noOutlist)

In [21]:
tar = knn_average(tps,noOutlots,distmatrix,noOutlist, all_tazs)
tar

,OBJECTID,taz,type,town,state,town_state,mpo,in_brmpo,subregion,Shape_Leng,...,TotalNN,MR_SumNN,MR_Avg_NN,MR_Avg_NN_2010,DR_SumNN,DR_Avg_NN,DR_Avg_NN_2010,HR_SumNN,HR_Avg_NN,HR_Avg_NN_2010
id,,,,,,,,,,,,,,,,,,,,,
4398,1,0,I,MIDDLEBOROUGH,MA,"MIDDLEBOROUGH,MA",SRPEDD,0,None,22284.463348,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2571,2,1,I,BRIDGEWATER,MA,"BRIDGEWATER,MA",OCPC,0,None,18241.697275,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2669,3,2,I,HALIFAX,MA,"HALIFAX,MA",OCPC,0,None,17900.674759,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4392,4,3,I,MIDDLEBOROUGH,MA,"MIDDLEBOROUGH,MA",SRPEDD,0,None,23735.098952,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2641,5,49,I,MARSHFIELD,MA,"MARSHFIELD,MA",BRMPO,1,SSC,19458.111922,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,5835,5834,I,TRURO,MA,"TRURO,MA",CCC,0,None,20079.041945,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4795,5836,5835,I,TRURO,MA,"TRURO,MA",CCC,0,None,25698.527422,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4794,5837,5836,I,PROVINCETOWN,MA,"PROVINCETOWN,MA",CCC,0,None,2836.977271,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Exports

In [22]:
tar.drop("geometry",axis=1).to_csv("J:\\Shared drives\\TMD_TSA\\Data\\Parking\\WebScraped_ParkingCost\\tazs_avg_rates2010_Apr25a.csv")

tar.to_file("J:\Shared drives\\TMD_TSA\Data\Parking\WebScraped_ParkingCost\\tazs_avg_ratesApr25a.geojson")  